# Coursera Capstone Project
### This notebook will be use to complete the final project of the IBM Data Science Professional Certificate

# Step 1: Set up Notebook

In [2]:
print("HELLO CAPSTONE PROJECT COURSE")

HELLO CAPSTONE PROJECT COURSE


In [1]:
import pandas as pd
import numpy as np

import pandas as pd # library for data analsysis

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         237 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0        conda-forge
    geopy:         1.20.0-py_0      conda-forge

The following pac

In [4]:
! pip install beautifulsoup4

     |████████████████████████████████| 102kB 10.9MB/s ta 0:00:01


In [5]:
! pip install lxml

     |████████████████████████████████| 5.8MB 30.5MB/s eta 0:00:01


In [6]:
! pip install request

  Stored in directory: /home/jupyterlab/.cache/pip/wheels/30/84/5f/484cfba678967ef58c16fce6890925d5c7172622f20111fbfd
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/c1/e3/c1/d02c8c58538853e4c9b78cadb74f6d5c5c370b48a69a7271aa
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/c3/c3/24/b5c132b537ab380c02d69e6bd4dec1f5db56b5fe19030473d7
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/d6/a4/78/01b20a9dc224dcc009fab669f7f27b943b8889c5150bd68d8a
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/23/7c/6e/f5b4e09d6596c8b8802b347e48f149031e2363368048f1347a
Successfully built request get post query-string public


In [7]:
from bs4 import BeautifulSoup as BS
import requests as r

# Step 2: Retrieve the data from Wikipedia

#### Here, Beautiful Soup is used to find the needed table from Wikipedia and placed in a temporary dataframe called 'n_list'

In [8]:
res = r.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BS(res.content,'lxml')
table = soup.find_all('table')[0]
n_list = pd.read_html(str(table))
n_list=n_list[0]
n_list.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### We remove all 'Not assigned' boroughs

In [9]:
n_list.set_index('Borough', inplace = True)
n_list.drop("Not assigned",axis=0, inplace = True)
n_list.reset_index(inplace=True)
n_list.set_index('Postcode', inplace = True)
n_list.reset_index(inplace=True)
n_list.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


#### The dataframe is now grouped based on the Postcode, combining the Neighbourhoods accordingly. We use commas to separate them.

In [10]:
n_list.sort_values("Postcode", ascending = True, inplace = True)
n_df=n_list.set_index('Postcode').groupby(level=['Postcode'],sort=False).agg(','.join)
n_df=n_df.reset_index()
n_dictionary=n_list[['Postcode','Borough']].set_index('Postcode').to_dict()
n_df['Borough'] = n_df.Postcode.map(n_dictionary['Borough'])
n_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Step 3: Clean the data

#### There is one "Not assigned" neighbourhood which will be replaced by the name of its borough.

In [11]:
n_df[n_df.Neighbourhood=='Not assigned']

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


In [12]:
# because it's only one, we can complete this by simply replacing the value manually
n_df['Neighbourhood']=n_df.Neighbourhood.str.replace("Not assigned", "Queen's Park")

In [13]:
# We can check this record using its index (85)
n_df.iloc[[85]]

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


#### Let's check our total number of rows

In [15]:
print("The number of rows in this dataframe is",n_df.shape[0])

The number of rows in this dataframe is 103


# Step 4: Add Geospatial Coordinates

#### Moving on, we can use the csv file, "Geospatial_data" provided by Coursera to create a dataframe of the Postal codes and their coordinates.

In [16]:
url = 'http://cocl.us/Geospatial_data'
geodata = pd.read_csv(url)
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### It's helpful if the naming conventions for our dataframes be consistant (i.e. "Postcode"; not "Postal Code")

In [17]:
geodata.rename(columns={'Postal Code': 'Postcode'},inplace=True)
geodata.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
geolat_dictionary=geodata[['Postcode','Latitude']].set_index('Postcode').to_dict()
geolat_dictionary=geolat_dictionary['Latitude']
geolong_dictionary=geodata[['Postcode','Longitude']].set_index('Postcode').to_dict()
geolong_dictionary=geolong_dictionary['Longitude']
n_df['Latitude'] = n_df.Postcode.map(geolat_dictionary)
n_df['Longitude'] = n_df.Postcode.map(geolong_dictionary)
n_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Richview Gardens,Kingsview Village,St. Phillip...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437


# Step 5: Explore the data

#### Below is a map of all of the data

In [20]:
address = 'Toronto,Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)


for lat, lng, borough, neighborhood in zip(n_df['Latitude'], n_df['Longitude'], n_df['Borough'], n_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=10,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Because I am specifically interested in Toronto and not the boroughs near, but outside of Toronto, I will create a dataframe more specific to Toronto.

In [21]:
#Let's look at a list of all of the borough to identify those to omit
Boroughs=n_df['Borough'].unique().tolist()
Boroughs

['Scarborough',
 'North York',
 'East York',
 'East Toronto',
 'Central Toronto',
 'Downtown Toronto',
 'York',
 'West Toronto',
 "Queen's Park",
 'Mississauga',
 'Etobicoke']

#### Now, we an create a duplicate dataframe called "toronto_df" and drop the boroughs outside of the core of Toronto. That is, we'll keep those boroughs with "Toronto" in the title.

In [22]:
non_Toronto_boroughs=['Scarborough','North York','East York','York',"Queen's Park",'Mississauga','Etobicoke']
toronto_df=n_df
drop_index=toronto_df[toronto_df.Borough.isin(non_Toronto_boroughs)].index
toronto_df.drop(drop_index,inplace=True)
print(toronto_df.shape)
toronto_df.head()

(38, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


#### Let's review this data on the map.

In [25]:
address = 'Toronto,Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


map_toronto_proper = folium.Map(location=[latitude, longitude], zoom_start=10)


for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=10,
        parse_html=False).add_to(map_toronto_proper)  
    
map_toronto_proper

#### Similar to the previous lab, we can review the characteristic of the neighbourhoods as it relates to the venues in them using Foursquare.

In [26]:
CLIENT_ID = 'PCPTC34PE31KFVHCDLZ3DCF4ZAI1RUYTMX30UDLKZDVFEAJQ' # your Foursquare ID
CLIENT_SECRET = 'WEVYX1BVPUFNMHA2RXOAVLQY1O44LJ0D2I2IGMR403PB4TOV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PCPTC34PE31KFVHCDLZ3DCF4ZAI1RUYTMX30UDLKZDVFEAJQ
CLIENT_SECRET:WEVYX1BVPUFNMHA2RXOAVLQY1O44LJ0D2I2IGMR403PB4TOV


#### Here, we retrieve the venue data from Foursquare

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
  
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
South Hill,Rathnelly,Summerhill West,Deer Park,Forest Hill SE
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Regent Park,Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
King,Adelaide,Richmond
Toronto Islands,Harbourfront East,Union Station
Toronto Dominion Centre,Design Exchange
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,Yorkville,North Midtown
University of Toronto,Harbord
Kensington Market,Grange Park,Chinatown
Railway Lands,South Niagara,Harbourfront West,Island airport,Bathurst Quay,CN Tower,King and Spadina
Stn A PO Boxes 25 The Esplanade
Underground city,First Canadian Place
Christie
Dovercourt Village,Dufferin
Trinity,Little Portugal
Exhibition Place,Brockton,Parkdale Village
High Park,The Junction South
Roncesvalles,Parkda

In [34]:
print(toronto_venues.shape)
toronto_venues.head()

(1707, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


# Step 6: Transform the data for cluster analysis

In [37]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,55,55,55,55,55,55
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"Cabbagetown,St. James Town",45,45,45,45,45,45
Central Bay Street,85,85,85,85,85,85
Christie,16,16,16,16,16,16
Church and Wellesley,91,91,91,91,91,91
"Commerce Court,Victoria Hotel",100,100,100,100,100,100
Davisville,38,38,38,38,38,38
Davisville North,9,9,9,9,9,9


In [38]:
print('There are {} uniques categories.'.format(toronto_venues['Venue Category'].unique().shape[0]))

There are 236 uniques categories.


In [39]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
toronto_onehot.shape

(1707, 237)

In [41]:
t_onehot_group = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
print(t_onehot_group.shape)
t_onehot_group.head()

(38, 237)


,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.018182,0.0,0.0,0.000000,0.0,0.0,0.000000
1,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.062500
2,"Cabbagetown,St. James Town",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
3,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.011765,0.0,...,0.0,0.0,0.0,0.011765,0.0,0.0,0.011765,0.0,0.0,0.011765
4,Christie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000


#### Let's look at the top venues in each neighbourhood

In [40]:
num_top_venues = 3

for hood in t_onehot_group['Neighbourhood']:
    print("----",hood,"----")
    temp = t_onehot_group[t_onehot_group['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[2:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Berczy Park ----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2  Italian Restaurant  0.04


---- Business Reply Mail Processing Centre 969 Eastern ----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2       Auto Workshop  0.06


---- Cabbagetown,St. James Town ----
                venue  freq
0         Coffee Shop  0.09
1         Pizza Place  0.04
2  Italian Restaurant  0.04


---- Central Bay Street ----
                venue  freq
0         Coffee Shop  0.14
1  Italian Restaurant  0.05
2                Café  0.05


---- Chinatown,Kensington Market,Grange Park ----
                           venue  freq
0                           Café  0.07
1  Vegetarian / Vegan Restaurant  0.06
2                            Bar  0.05


---- Christie ----
           venue  freq
0           Café  0.19
1  Grocery Store  0.19
2           Park  0.12


---- Church and Wellesley ----
                 venue  freq
0          Coff

# Step 7: Cluster Analysis

#### K-Means model is used to create 3 cluster labels for the data. 

In [86]:
# set number of clusters
kclusters = 4
#set number of trials
n_init = 100

toronto_clustering = t_onehot_group.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0, n_init=n_init).fit(toronto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 3, 2, 2, 2, 2, 0,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

#### We can look at this information tabularly and diagramatically. Let's create a dataframe with the top 10 venues in each neighbourhood.

In [87]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [88]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = t_onehot_group['Neighbourhood']

for ind in np.arange(t_onehot_group.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(t_onehot_group.iloc[ind, :], num_top_venues)
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighbourhoods_venues_sorted.head()

toronto_merged = toronto_df
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
print(toronto_merged.shape)
toronto_merged.head()

(38, 16)


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Pub,Neighborhood,Health Food Store,Other Great Outdoors,Trail,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Ice Cream Shop,Pizza Place,Brewery,Bubble Tea Shop,Restaurant
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,2,Park,Pet Store,Burrito Place,Sushi Restaurant,Brewery,Ice Cream Shop,Pub,Fish & Chips Shop,Movie Theater,Steakhouse
43,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Yoga Studio,Chinese Restaurant,Stationery Store,Clothing Store,Diner
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Construction & Landscaping,Park,Swim School,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


#### The result of our cluster analysis is in the below map and tables

In [89]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=15,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### There seems to be a lot of homogeneous neighbourhoods in Toronto!

#### Here are the lists of neighbourhoods and their clusters.

### Cluster 1

In [90]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,Lawrence Park,0,Construction & Landscaping,Park,Swim School,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
50,Downtown Toronto,Rosedale,0,Park,Trail,Playground,Building,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Yoga Studio
64,Central Toronto,"Forest Hill North,Forest Hill West",0,Trail,Jewelry Store,Sushi Restaurant,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [91]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,Roselawn,1,Garden,Yoga Studio,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [92]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,The Beaches,2,Pub,Neighborhood,Health Food Store,Other Great Outdoors,Trail,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run
41,East Toronto,"The Danforth West,Riverdale",2,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Ice Cream Shop,Pizza Place,Brewery,Bubble Tea Shop,Restaurant
42,East Toronto,"The Beaches West,India Bazaar",2,Park,Pet Store,Burrito Place,Sushi Restaurant,Brewery,Ice Cream Shop,Pub,Fish & Chips Shop,Movie Theater,Steakhouse
43,East Toronto,Studio District,2,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Yoga Studio,Chinese Restaurant,Stationery Store,Clothing Store,Diner
45,Central Toronto,Davisville North,2,Pizza Place,Breakfast Spot,Hotel,Food & Drink Shop,Dog Run,Park,Clothing Store,Sandwich Place,Gym,Antique Shop
46,Central Toronto,North Toronto West,2,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Chinese Restaurant,Dessert Shop,Café,Rental Car Location,Restaurant,Diner
47,Central Toronto,Davisville,2,Sandwich Place,Dessert Shop,Pizza Place,Gym,Coffee Shop,Café,Sushi Restaurant,Italian Restaurant,Thai Restaurant,Flower Shop
49,Central Toronto,"South Hill,Rathnelly,Summerhill West,Deer Park...",2,Coffee Shop,Pub,Bagel Shop,Light Rail Station,Supermarket,Vietnamese Restaurant,Sushi Restaurant,Pizza Place,Restaurant,American Restaurant
51,Downtown Toronto,"Cabbagetown,St. James Town",2,Coffee Shop,Pub,Pizza Place,Italian Restaurant,Park,Restaurant,Café,Bakery,Jewelry Store,Japanese Restaurant
52,Downtown Toronto,Church and Wellesley,2,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Gym


In [93]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,"Moore Park,Summerhill East",3,Gym,Playground,Restaurant,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
